<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/chat_engine/chat_engine_context_citations_cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cohere Chat Engine with citations and documents - Usage Example
========================================
    
This notebook demonstrates how to use the Cohere chat engine to generate responses to user input with citations and context documents(Cohere documents mode - see the documentation [here](https://docs.cohere.com/docs/retrieval-augmented-generation-rag) and [here](https://docs.cohere.com/docs/retrieval-augmented-generation-rag). Cohere Chat Engine provides several modes for generating responses, including: Chat, Async Chat, Stream Chat and Async Stream Chat. We will use a vector index to retrieve the context documents and the next sample [data](https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt) 



Download Data
========================================

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

Install requirements
========================================


In [ ]:
%pip install llama-index
%pip install llama-index-llms-cohere
%pip install llama-index-embeddings-cohere
%pip install cohere

Let's start by creating a new index and adding the context documents into it.
========================================

### Imports

In [ ]:
import os
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.cohere import Cohere
from llama_index.core import Settings

### Constants

In [ ]:
os.environ["COHERE_API_KEY"] = "YOUR_COHERE_API_KEY_HERE"
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

### Create a new Cohere LLM instance and register it in settings

In [ ]:
LLM = Cohere(
    "command",
    api_key=COHERE_API_KEY,
    temperature=0.5,
    additional_kwargs={"prompt_truncation": "AUTO"},
)

### Create a new Cohere embedding

In [ ]:
def get_cohere_embed_model(
    input_type="search_document", model_name="embed-english-v3.0"
):
    cohere_embed_model = CohereEmbedding(
        cohere_api_key=COHERE_API_KEY,
        model_name=model_name,
        input_type=input_type,
    )

    return cohere_embed_model

### Create a new service context

In [ ]:
def set_cohere_service_context(input_type="search_document"):
    Settings.llm = LLM
    Settings.embed_model = get_cohere_embed_model(input_type=input_type)

### Read the sample documents from the file

In [ ]:
def get_sample_documents(path="./data/paul_graham/"):
    sample_documents = SimpleDirectoryReader(input_dir=path).load_data()
    return sample_documents

### For the Chat Engine we should use Cohere embed model with input type "search_query". See the [documentation](https://docs.cohere.com/reference/embed) for more details. So we need to use a new index with the input type "search_query"

In [ ]:
def get_opensearch_query_index():
    index = VectorStoreIndex.from_documents(
        get_sample_documents(),
        embed_model=get_cohere_embed_model(input_type="search_query"),
    )
    return index

### And finally, we can create a new chat engine using Cohere Context mode

In [ ]:
def get_chat_engine_by_index_and_mode(index, mode="citations_context"):
    chat_engine = index.as_chat_engine(chat_mode=mode, llm=LLM)
    return chat_engine

## Let's run it and see the results
### Chat

In [ ]:
# Set service context using Settings
set_cohere_service_context()
# fill_opensearch_documents_index()
# Create a new chat engine
chat_engine = get_chat_engine_by_index_and_mode(get_opensearch_query_index())

# Ask a question using the chat engine
question = "What did Paul Graham do with SHRDLU?"
llm_response = chat_engine.chat(question)
# Print the response
print(llm_response)
# Print the citations
print(f"Citations: {llm_response.citations}")
# Print the context documents
print(f"Documents: {llm_response.documents}")

Paul Graham reverse-engineered SHRDLU as an undergraduate for his thesis. SHRDLU was a PBS documentary that showed Terry Winograd using SHRDLU, and Paul Graham believed that it was already climbing the lower slopes of intelligence. He said that in the 80s, it was a stimulating bit of code and he enjoyed working with it. He was fascinated by the idea that if SHRDLU's vocabulary was expanded, it would quickly learn to mimic human language.
Citations: [Citation(text='reverse-engineered SHRDLU', start=12, end=37, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='PBS documentary', start=87, end=102, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='Terry Winograd using SHRDLU', start=115, end=142, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='climbing the lower slopes of intelligence.', start=189, end=231, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='stimulating bit of code', start=266, end=289, doc

### Async Chat

In [ ]:
llm_response = await chat_engine.achat(question)
print(llm_response)
print(f"Citations: {llm_response.citations}")
print(f"Documents: {llm_response.documents}")

For his undergraduate thesis, Paul Graham reverse-engineered SHRDLU. He later realised that the AI practices at the time were a hoax and that there was an unbridgeable gap between what they could do and understanding natural language.
Citations: [Citation(text='undergraduate thesis', start=8, end=28, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='reverse-engineered SHRDLU.', start=42, end=68, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='the AI practices at the time were a hoax', start=92, end=132, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='unbridgeable gap', start=155, end=171, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='understanding natural language.', start=203, end=234, document_ids=['7451555d-c245-42c1-90fe-180df5af233c'])]
Documents: [Document(id='7451555d-c245-42c1-90fe-180df5af233c', text='I couldn\'t have put this into words when I was 18. All I knew at the time was that I 

### Stream Chat 

In [ ]:
llm_response = chat_engine.stream_chat(question)
llm_response.print_response_stream()

print("Citations: ")
print(llm_response.citations)

print("Documents: ")
print(llm_response.documents)

For his undergraduate thesis, Paul Graham reverse-engineered SHRDLU and later discovered that the AI practices at the time were a hoax and that there was an unbridgeable gap between what they could do and understanding natural language.Citations: 
[Citation(text='undergraduate thesis', start=8, end=28, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='reverse-engineered SHRDLU', start=42, end=67, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='AI practices at the time were a hoax', start=98, end=134, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='unbridgeable gap between what they could do and understanding natural language.', start=157, end=236, document_ids=['7451555d-c245-42c1-90fe-180df5af233c'])]
Documents: 
[Document(id='7451555d-c245-42c1-90fe-180df5af233c', text='I couldn\'t have put this into words when I was 18. All I knew at the time was that I kept taking philosophy courses and they kept being boring. So I 

### Async Stream Chat

In [ ]:
llm_response = await chat_engine.astream_chat(question)
await llm_response.aprint_response_stream()
print("Citations: ")
print(llm_response.citations)
print("Documents: ")
print(llm_response.documents)

Paul Graham reverse-engineered SHRDLU as an undergraduate for his thesis. However, he later realised that the AI practices at the time were a hoax and that there was an unbridgeable gap between what they could do and understanding natural language. After this, he refocused on Lisp and wrote a book on it called On Lisp, which wasn't published until 1993.Citations: 
[Citation(text='reverse-engineered SHRDLU', start=12, end=37, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='undergraduate', start=44, end=57, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='thesis.', start=66, end=73, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='AI practices at the time were a hoax', start=110, end=146, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='unbridgeable gap', start=169, end=185, document_ids=['7451555d-c245-42c1-90fe-180df5af233c']), Citation(text='understanding natural language.', start=217, end=248, do